<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/master/MMVCTrainerFrontendDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# このノートブックは管理されていません。
# VCClientをColabで使用する場合は、次のノートブックをご使用ください。

# This notebook is not being managed.
# If you're using VCClient in Colab, please use the following notebook.

https://github.com/w-okada/voice-changer/blob/master/Realtime_Voice_Changer_on_Colab.ipynb




Voice Changer Simple (デモ版)
---

このノートはVoice ChangerをColab上で動かすデモ版です。

正式版はローカルPCのDocker上で動かすアプリケーションです。

正式版は、多くの場合より少ないタイムラグで滑らかに音声を変換できます。

詳細な使用方法はこちらの[リポジトリ](https://github.com/w-okada/voice-changer)からご確認ください。


# GPUを確認
GPUを用いたほうが高速に処理が行えます。

下記のコマンドでGPUが確認できない場合は、上のメニューから

「ランタイム」→「ランタイムの変更」→「ハードウェア アクセラレータ」

でGPUを選択してください。

In [ ]:
# (1) GPUの確認
!nvidia-smi

Sat Dec 10 23:45:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    31W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# リポジトリのクローン
リポジトリをクローンします

In [ ]:
# (2) リポジトリのクローン
!git clone --depth 1 https://github.com/w-okada/voice-changer.git -b dev
%cd voice-changer/demo/

!cp ../template/setting_mmvc_colab.json ../frontend/dist/assets/setting.json

Cloning into 'voice-changer'...
remote: Enumerating objects: 1029, done.
remote: Counting objects: 100% (1029/1029), done.
remote: Compressing objects: 100% (919/919), done.
remote: Total 1029 (delta 21), reused 979 (delta 10), pack-reused 0
Receiving objects: 100% (1029/1029), 71.87 MiB | 26.18 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/voice-changer/demo


# モジュールのインストール

必要なモジュールをインストールします。

In [ ]:
# (3) 設定ファイルの確認
!apt-get install -y espeak libsndfile1-dev &> /dev/null
!pip install unidecode &> /dev/null
!pip install phonemizer &> /dev/null
!pip install retry &> /dev/null
!pip install python-socketio &> /dev/null
!pip install fastapi &> /dev/null
!pip install python-multipart &> /dev/null
!pip install uvicorn &> /dev/null
!pip install websockets &> /dev/null
!pip install pyOpenSSL &> /dev/null
!pip install pyopenjtalk==0.2.0 &> /dev/null

%cd MMVC_Trainer/monotonic_align/
!python setup.py build_ext --inplace
%cd ../../

/content/voice-changer/demo/MMVC_Trainer/monotonic_align
running build_ext
building 'monotonic_align.core' extension
creating build
creating build/temp.linux-x86_64-3.8
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.8/dist-packages/numpy/core/include -I/usr/include/python3.8 -c core.c -o build/temp.linux-x86_64-3.8/core.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 -Wl,-Bsymbolic-functions -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/core.o -o /content/voice-changer/demo/MMVC_Trainer/monotonic_align/monotonic_align/core.cpython-38-x86_64-linux-gnu.so
/content/voice-changer/demo


# サーバの起動

サーバを起動します。(4-1)

サーバの起動状況を確認します。(4-2)

このセルは繰り返し実行することになるのでCtrl+Retでセルを実行してください。

下記のようなテキストが表示されたら起動完了です。

**`DEBUG:asyncio:Using selector: EpollSelector`**

```
    Phase name:__main__
    PHASE3:__main__
    PHASE1:__main__
Start MMVC SocketIO Server
    CONFIG:None, MODEL:None
DEBUG:asyncio:Using selector: EpollSelector
```



In [ ]:
# (4-1) サーバの起動
import random
PORT = 10000 + random.randint(1, 9999)
LOG_FILE = f"LOG_FILE_{PORT}"

get_ipython().system_raw(f'python3 MMVCServerSIO.py -p {PORT} --colab True >{LOG_FILE} 2>&1 &')
#print(f"PORT:{PORT}, LOG_FILE:{LOG_FILE}")

In [ ]:
# (4-2) サーバの起動確認
!sleep 5
!tail -20 {LOG_FILE}

    Phase name:__main__
    PHASE3:__main__
    PHASE1:__main__
Start MMVC SocketIO Server
    CONFIG:None, MODEL:None
DEBUG:asyncio:Using selector: EpollSelector


# プロキシを起動
ウェブサーバへのアクセスをするためのプロキシを起動します。

表示されたURLをクリックして開くと別タブでアプリが開きます。

Colabなので、ロードにある程度時間がかかります(30秒くらい)。

In [ ]:
# (5) プロキシを起動
from google.colab.output import eval_js
proxy = eval_js( "google.colab.kernel.proxyPort(" + str(PORT) + ")" )
print(f"{proxy}trainer/")

https://vljlzcgk39q-496ff2e9c6d22116-17357-colab.googleusercontent.com/trainer/
